In [16]:
from pathlib import Path
import joblib
from pprint import pprint

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    NormalizeCurrency,
    OrdinalColumnMapper,
)

pd.set_option('display.max_columns', None)

In [11]:
def count_frequent_values(series: pd.Series, min_count: int = 10) -> int:
    """
    Count the number of unique values in a series that appear more than a specified number of times.

    Parameters:
    series (pd.Series): The pandas Series to count values from.
    min_count (int): The minimum number of occurrences for a value to be counted. Default is 10.

    Returns:
    int: The number of unique values that appear more than min_count times.
    """
    vc = series.value_counts()
    vc = vc[vc > min_count]
    return len(vc)

In [2]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"
assert DATA_DIR.exists()

DATA_PATH = DATA_DIR / "base_indices_2005-2023.csv"

In [3]:
df = joblib.load(DATA_DIR / "base_indices.pkl")
df = df.reset_index()
df = df[df["Promedio Puntaje (promedio matemáticas y lenguaje)"].notna()].reset_index(
    drop=True
)
df = (
    df[df["Pregrado/Posgrado"] == "Pregrado"]
    .drop(columns=["Pregrado/Posgrado"])
    .reset_index(drop=True)
    .rename(columns={"Año": "Fecha"})
)
df["Año"] = df["Fecha"].dt.year
df["Grado Académico"] = (
    df["Grado Académico"]
    .str.strip()
    .str.split(" ")
    .str[0]
    .str.replace(r'(Licencia).*', 'Licencia', regex=True)
    .str.replace('LLicencia', 'Licencia', regex=False)
    .str.replace(r'(Bachiller).*', 'Bachiller', regex=True)
    .fillna("Desconocido")
    .str.replace("No", "No aplica", regex=False)
)

In [4]:
# df.info()

In [5]:
# nulls = df.isnull().sum() / len(df) * 100
# nulls.sort_values(ascending=False)

In [6]:
data_pipeline = Pipeline(
    [
        ("drop_columns", DropColumns()),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.24)),
        ("preprocess_tipo_moneda", NormalizeCurrency()),
        ("ordinal_encoder", OrdinalColumnMapper()),
    ]
)
data_pipeline

Pipeline(steps=[('drop_columns', DropColumns()),
                ('drop_high_na', DropHighNAPercentage(na_threshold=0.24)),
                ('preprocess_tipo_moneda', NormalizeCurrency()),
                ('ordinal_encoder', OrdinalColumnMapper())])

In [7]:
processed_df = data_pipeline.fit_transform(df)

In [8]:
processed_df.head()

,Fecha,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Comuna donde se imparte la carrera o programa,Nombre Region,Cód. Carrera,Carrera Genérica,Nombre Programa,Horario,Tipo Programa,Area Conocimiento,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Nombre del Campus,Duración (en semestres),Cód. Sede,Título,Grado Académico,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Año
0,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,7444,Administración pública y similares,Administración Pública,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1954,Andrés Bello,11.0,1001001,Administrador Público,1,702.0,919.0,758.0,708.0,129.0,177300.0,4861900.0,121000.0,130.0,68.0,82.0,150,302.0,392.0,694,2023
1,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,27288,"Contabilidad, Auditoría y similares",Contador Auditor,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2013,Andrés Bello,10.0,1001001,Contador Auditor,1,766.0,885.0,750.0,704.0,84.0,177300.0,6137900.0,121000.0,70.0,54.0,45.0,99,191.0,205.0,396,2023
2,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,7420,Ingeniería Comercial,Ingeniería Comercial,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1924,Andrés Bello,10.0,1001001,Ingeniero Comercial,1,863.0,963.0,838.0,803.0,419.0,177300.0,6844100.0,121000.0,380.0,353.0,234.0,587,1535.0,1126.0,2661,2023
3,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,Santiago,Región Metropolitana,27287,Ingeniería en Control de Gestión y similares,Ingeniería en Información y Control de Gestión,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2013,Andrés Bello,10.0,1001001,Ingeniero en Información y Control de Gestión,1,771.0,881.0,752.0,701.0,172.0,177300.0,6137900.0,121000.0,140.0,101.0,86.0,187,395.0,358.0,753,2023
4,2023-01-01,1001,U. DE CHILE,0,0,0,0,0,0,0,La Pintana,Región Metropolitana,7417,Ingeniería Agronómica,Ingeniería Agronómica,Diurno,Programa Regular,"Agricultura, Silvicultura, Pesca y Veterinaria",Profesional con Licenciatura,Ingreso Directo,1915,Sur,10.0,1001001,Ingeniero Agrónomo,1,649.0,830.0,674.0,603.0,129.0,177300.0,5681600.0,121000.0,130.0,76.0,56.0,132,378.0,268.0,646,2023


In [23]:
processed_df[['Título', 'Area Conocimiento']]

,Título,Area Conocimiento
0,Administrador Público,Administración y Comercio
1,Contador Auditor,Administración y Comercio
2,Ingeniero Comercial,Administración y Comercio
3,Ingeniero en Información y Control de Gestión,Administración y Comercio
4,Ingeniero Agrónomo,"Agricultura, Silvicultura, Pesca y Veterinaria"
...,...,...
39721,Técnico Nivel Sup. Diseñador Gráfico y Multimedia,Arte y Arquitectura
39722,Técnico Nivel Sup. en Gestión y Soporte de Redes,Tecnología
39723,Técnico Nivel Sup. Analista de Sistemas,Tecnología
39724,Técnico Nivel Sup. Analista de Sistemas,Tecnología


In [19]:
pprint(processed_df["Tipo Carrera"].unique().tolist())
    

['Profesional con Licenciatura',
 'Licenciatura',
 'Bachillerato',
 'Plan Común o Ciclo Básico',
 'Profesional',
 'Técnico Nivel Superior']


In [14]:
count_frequent_values(processed_df["Título"], min_count=25)

202

In [40]:
vc = processed_df['Grado Académico'].value_counts()
vc.index.str.strip().str.split(' ').str[0].value_counts()

Grado Académico
Licenciado               507
Bachiller                 63
Licenciado(a)             61
Licenciatura              50
Licenciado/A              10
Licenciado(A)              7
Bachillerato               2
Licenciada/Licenciado      2
Licenciado/a               2
Licencia                   1
No                         1
Preparador                 1
Ingeniero                  1
Licenciada(o)              1
Licenciada                 1
LLicenciado                1
Licenciado(a)En            1
Name: count, dtype: int64

In [26]:
joblib.dump(processed_df, DATA_DIR / "processed_df.pkl")

['/home/pabloskewes/FCFM/Minería de Datos/proyecto-mineria-de-datos/data/processed_df.pkl']

In [10]:
processed_df[processed_df["Nombre del Campus"].isnull()]

,Año,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Comuna donde se imparte la carrera o programa,Nombre Region,Cód. Carrera,Carrera Genérica,Nombre Programa,Horario,Tipo Programa,Area Conocimiento,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Nombre del Campus,Duración (en semestres),Título,Grado Académico,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total
116,2023-01-01,1003,U. DE CONCEPCIÓN,0,0,1,0,0,0,0,Concepción,Región del Bío-Bío,38520,Administración pública y similares,Administración Pública y Ciencia Política,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2018,NaN,10.0,Administrador Público,Licenciado en Ciencias Políticas y Administrat...,688.0,949.0,778.0,691.0,72.0,150000.0,5582083.0,39500.0,70.0,34.0,45.0,79,208.0,208.0,416
117,2023-01-01,1003,U. DE CONCEPCIÓN,0,0,1,0,0,0,0,Concepción,Región del Bío-Bío,1361,"Contabilidad, Auditoría y similares",Auditoría,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1968,NaN,8.0,Contador Auditor,Licenciado en Contabilidad y Auditoría,643.0,884.0,692.0,617.0,60.0,150000.0,5582083.0,39500.0,60.0,35.0,25.0,60,144.0,128.0,272
118,2023-01-01,1003,U. DE CONCEPCIÓN,0,0,1,0,0,0,0,Los Ángeles,Región del Bío-Bío,1411,"Contabilidad, Auditoría y similares",Auditoría,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1978,NaN,8.0,Contador Auditor,Licenciado en Contabilidad y Auditoría,605.0,811.0,635.0,560.0,41.0,150000.0,5582083.0,39500.0,40.0,24.0,17.0,41,74.0,71.0,145
119,2023-01-01,1003,U. DE CONCEPCIÓN,0,0,1,0,0,0,0,Chillán,Región de Ñuble,1532,Ingeniería Comercial,Ingeniería Comercial,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,2005,NaN,10.0,Ingeniero Comercial,Licenciatura en Ciencias Económicas o en Cienc...,680.0,859.0,718.0,640.0,60.0,150000.0,5582083.0,39500.0,60.0,41.0,22.0,63,184.0,110.0,294
120,2023-01-01,1003,U. DE CONCEPCIÓN,0,0,1,0,0,0,0,Concepción,Región del Bío-Bío,1372,Ingeniería Comercial,Ingeniería Comercial,Diurno,Programa Regular,Administración y Comercio,Profesional con Licenciatura,Ingreso Directo,1957,NaN,10.0,Ingeniero Comercial,Licenciatura en Ciencias Económicas o en Cienc...,733.0,909.0,772.0,698.0,132.0,150000.0,5582083.0,39500.0,130.0,81.0,64.0,145,311.0,252.0,563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39721,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,2,3,5,1,0,1,2,Santiago,Región Metropolitana,4988,Técnico en Diseño Gráfico y similares,TNS en Diseño Gráfico y Multimedia,Vespertino,Programa Regular,Arte y Arquitectura,Técnico Nivel Superior,Ingreso Directo,2005,NaN,7.0,Técnico Nivel Sup. Diseñador Gráfico y Multimedia,NaN,470.0,NaN,NaN,NaN,1.0,47000.0,930000.0,232500.0,50.0,10.0,3.0,13,10.0,3.0,13
39722,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,2,3,5,1,0,1,2,Santiago,Región Metropolitana,4981,Técnico en Administración de Redes Computacion...,TNS en Gestión y Soporte de Redes,Diurno,Programa Regular,Tecnología,Técnico Nivel Superior,Ingreso Directo,2002,NaN,7.0,Técnico Nivel Sup. en Gestión y Soporte de Redes,NaN,580.0,NaN,NaN,NaN,2.0,47000.0,930000.0,232500.0,50.0,9.0,0.0,9,55.0,0.0,55
39723,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,2,3,5,1,0,1,2,Santiago,Región Metropolitana,4983,Técnico en Análisis de Sistemas Informáticos,TNS en Análisis de Sistemas,Diurno,Progra

In [9]:
nulls = processed_df.isnull().sum() / len(df) * 100
nulls.sort_values(ascending=False)

Nombre del Campus                                     38.264613
Puntaje de corte (promedio de la carrera)             15.903942
Puntaje de corte (primer seleccionado)                15.659769
Valor del Título                                      13.948044
Grado Académico                                       13.389216
Puntaje de corte (último seleccionado)                10.675628
Título                                                 5.789659
Vacantes                                               3.408347
Valor de matrícula                                     1.092484
Matrícula primer año hombres                           0.999346
Nº Alumnos Ingreso Via PSU o PDT                       0.979208
Valor de arancel                                       0.533656
Matrícula total hombres                                0.483311
Matrícula primer año mujeres                           0.435483
Duración (en semestres)                                0.229069
Matrícula total mujeres                 

In [11]:
processed_df[processed_df['Tipo Moneda'] == 'Uf'][['Valor de matrícula', 'Valor del Título']]

,Valor de matrícula,Valor del Título
2231,842545.43,0.0
2232,NaN,0.0
2233,NaN,0.0
2234,842545.43,0.0
2235,769280.61,0.0
...,...,...
191076,NaN,NaN
191077,NaN,NaN
191079,439588.92,NaN
191080,109897.23,NaN


,Año,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,...,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula primer año extranjeros,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula total extranjeros,Matrícula Total,Códgo SIES,Pregrado/Posgrado
0,2023-01-01,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,68.0,82.0,2.0,150,302.0,392.0,5.0,694,I70S1C1J1V2,Pregrado
1,2023-01-01,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,54.0,45.0,10.0,99,191.0,205.0,23.0,396,I70S1C814J1V1,Pregrado
2,2023-01-01,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,353.0,234.0,16.0,587,1535.0,1126.0,52.0,2661,I70S1C58J1V1,Pregrado
3,2023-01-01,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,101.0,86.0,10.0,187,395.0,358.0,27.0,753,I70S1C816J1V1,Pregrado
4,2023-01-01,1001,U. DE CHILE,Univ.,(a) Universidades CRUCH,(a) Universidades Estatales CRUCH,(a) Acreditada,(a) Autónoma,(a) Adscritas a Gratuidad,(a) Subsistema Universitario,...,76.0,56.0,4.0,132,378.0,268.0,6.0,646,I70S1C47J1V1,Pregrado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182914,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,C.F.T.,(d) Centros de Formación Técnica,(f) Centros de Formación Técnica,(b) No Acreditada,(a) Autónoma,(b) No Adscritas/No Aplica,(c) No adscrito,...,10.0,3.0,0.0,13,10.0,3.0,0.0,13,I390S1C4J2V1,Pregrado
182915,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,C.F.T.,(d) Centros de Formación Técnica,(f) Centros de Formación Técnica,(b) No Acreditada,(a) Autónoma,(b) No Adscritas/No Aplica,(c) No adscrito,...,9.0,0.0,1.0,9,55.0,0.0,0.0,55,I390S1C3J1V1,Pregrado
182917,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,C.F.T.,(d) Centros de Formación Técnica,(f) Centros de Formación Técnica,(b) No Acreditada,(a) Autónoma,(b) No Adscritas/No Aplica,(c) No adscrito,...,6.0,0.0,0.0,6,19.0,1.0,0.0,20,I390S1C1J1V1,Pregrado
182918,2005-01-01,3025,C.F.T. INSTITUTO TECNOLÓGICO DE CHILE,C.F.T.,(d) Centros de Formación Técnica,(f) Centros de Formación Técnica,(b) No Acreditada,(a) Autónoma,(b) No Adscritas/No Aplica,(c) No adscrito,...,14.0,4.0,0.0,18,77.0,11.0,0.0,88,I390S1C1J2V1,Pregrado


Mínimo Puntaje Ranking                                91.340796
Máximo Puntaje Ranking                                91.334854
Promedio Puntaje Ranking                              91.269489
Mínimo Puntaje NEM                                    90.884158
Máximo Puntaje NEM                                    90.878216
Promedio Puntaje NEM                                  90.793654
Mención o Especialidad                                85.976332
Puntaje de corte (promedio de la carrera)             84.489859
Puntaje de corte (primer seleccionado)                84.445521
Puntaje de corte (último seleccionado)                83.515333
Mínimo Puntaje (promedio matemáticas y lenguaje)      81.913673
Promedio Puntaje (promedio matemáticas y lenguaje)    81.841452
Máximo Puntaje (promedio matemáticas y lenguaje)      81.707980
Nº Alumnos Ingreso Via PSU o PDT                      81.108272
Nº Alumnos Ingreso Otra Via                           68.050902
Grado Académico                         

In [13]:
uf_index = processed_df[processed_df['Tipo Moneda'] == 'Uf'].index
df.loc[uf_index, ['Valor de matrícula', 'Valor del Título']]

,Valor de matrícula,Valor del Título
2231,23.0,0.0
2232,NaN,0.0
2233,NaN,0.0
2234,23.0,0.0
2235,21.0,0.0
...,...,...
191076,NaN,NaN
191077,NaN,NaN
191079,12.0,NaN
191080,3.0,NaN


In [9]:

df[df['Tipo Moneda'] == 'Uf']

,Año,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,...,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula primer año extranjeros,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula total extranjeros,Matrícula Total,Códgo SIES,Pregrado/Posgrado


In [8]:
processed_df

,Año,Cód. Institución,Nombre Institución,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,...,Valor del Título,Tipo Moneda,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula total extranjeros,Matrícula Total,Pregrado/Posgrado
0,2023-01-01,1001,U. DE CHILE,Univ.,0,0,0,0,0,0,...,121000.0,Pesos,68.0,82.0,150,302.0,392.0,5.0,694,Pregrado
1,2023-01-01,1001,U. DE CHILE,Univ.,0,0,0,0,0,0,...,121000.0,Pesos,54.0,45.0,99,191.0,205.0,23.0,396,Pregrado
2,2023-01-01,1001,U. DE CHILE,Univ.,0,0,0,0,0,0,...,121000.0,Pesos,353.0,234.0,587,1535.0,1126.0,52.0,2661,Pregrado
3,2023-01-01,1001,U. DE CHILE,Univ.,0,0,0,0,0,0,...,121000.0,Pesos,101.0,86.0,187,395.0,358.0,27.0,753,Pregrado
4,2023-01-01,1001,U. DE CHILE,Univ.,0,0,0,0,0,0,...,121000.0,Pesos,76.0,56.0,132,378.0,268.0,6.0,646,Pregrado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218768,2005-01-01,4023,ACADEMIA POLITÉCNICA MILITAR,F.F.A.A.,4,6,1,4,1,3,...,0.0,Pesos,15.0,0.0,15,15.0,0.0,0.0,15,Posgrado
218769,2005-01-01,4023,ACADEMIA POLITÉCNICA MILITAR,F.F.A.A.,4,6,1,4,1,3,...,0.0,Pesos,10.0,0.0,10,10.0,0.0,1.0,10,Posgrado
218770,2005-01-01,4031,ACADEMIA DE GUERRA NAVAL,F.F.A.A.,4,6,1,4,1,3,...,0.0,Pesos,17.0,4.0,21,17.0,4.0,0.0,21,Posgrado
218771,2005-01-01,4031,ACADEMIA DE GUERRA NAVAL,F.F.A.A.,4,6,1,4,1,3,...,0.0,Pesos,18.0,8.0,26,18.0,8.0,0.0,26,Posgrado


In [30]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218773 entries, 0 to 218772
Data columns (total 34 columns):
 #   Column                                         Non-Null Count   Dtype         
---  ------                                         --------------   -----         
 0   Año                                            218773 non-null  datetime64[ns]
 1   Cód. Institución                               218773 non-null  int64         
 2   Nombre Institución                             218773 non-null  object        
 3   Tipo Institución                               218773 non-null  object        
 4   Clasificación1                                 218773 non-null  int64         
 5   Clasificación2                                 218773 non-null  int64         
 6   Clasificación3                                 218773 non-null  int64         
 7   Clasificación4                                 218773 non-null  int64         
 8   Clasificación5                              

In [16]:
processed_df['Tipo Moneda'].value_counts()

Tipo Moneda
Pesos    217446
Uf         1327
Name: count, dtype: int64

In [28]:


get_uf_value()


36632.41

In [12]:
pprint(processed_df.columns.tolist())

['Año',
 'Cód. Institución',
 'Nombre Institución',
 'Tipo Institución',
 'Clasificación1',
 'Clasificación2',
 'Clasificación3',
 'Clasificación4',
 'Clasificación5',
 'Clasificación6',
 'Comuna donde se imparte la carrera o programa',
 'Nombre Region',
 'Cód. Carrera',
 'Carrera Genérica',
 'Nombre Programa',
 'Horario',
 'Tipo Programa',
 'Area Conocimiento',
 'Tipo Carrera',
 'IngresoDirecto',
 'Año Inicio Actividades',
 'Duración (en semestres)',
 'Título',
 'Valor de matrícula',
 'Valor del Título',
 'Tipo Moneda',
 'Matrícula primer año hombres',
 'Matrícula primer año mujeres',
 'Matrícula Primer Año',
 'Matrícula total hombres',
 'Matrícula total mujeres',
 'Matrícula total extranjeros',
 'Matrícula Total',
 'Pregrado/Posgrado']


In [ ]:
def compute_kmeans(
    df: pd.DataFrame,
    cluster_range: Iterable[int],
) -> pd.DataFrame:
    """
    Perform k-means clustering for a range of k values and collect metrics.
    Args:
        df: Dataframe to cluster.
        cluster_range: Range of k values to try.
    Returns:
        Dictionary with k as key and silhouette score as value.
    """
    interia = []
    silhouette = []
    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42).fit(df)
        interia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(df, kmeans.labels_))

    return pd.DataFrame(
        {"inertia": interia, "silhouette": silhouette},
        index=cluster_range,
    )

In [ ]:
def plot_kmean_results(k_range, inertia_values, silhouette_scores):
    """
    Plot the results of the k-means clustering.
    Args:
        k_range: Range of k values used.
        inertia_values: List of inertia values for each k.
        silhouette_scores: List of silhouette scores for each k.
    """
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    ax[0].plot(k_range, inertia_values, marker="o")
    ax[0].set_xlabel("Número de clusters")
    ax[0].set_ylabel("Inercia")
    ax[0].set_title("Método del codo")

    ax[1].plot(k_range, silhouette_scores, marker="o")
    ax[1].set_xlabel("Número de clusters")
    ax[1].set_ylabel("Silhouette")
    ax[1].set_title("Método de la silueta")
    plt.show()

In [ ]:
kmeans_df = compute_kmeans(df, range(2, 21))